In [ ]:
import os
dry_run = False
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import sys
from pdb import set_trace
from contextlib import nullcontext

from lerobot.common.datasets.lerobot_dataset import LeRobotDataset
from lerobot.common.datasets.sampler import EpisodeAwareSampler
from lerobot.common.datasets.factory import make_dataset
from lerobot.common.policies.factory import make_policy
from lerobot.common.datasets.utils import cycle

from lerobot.common.policies.diffusion.configuration_diffusion import DiffusionConfig
from lerobot.common.policies.diffusion.modeling_diffusion import DiffusionPolicy

from tqdm.notebook import trange

from lerobot.common.utils.utils import format_big_number

import torch
from torch.cuda.amp import GradScaler

from aim import Run

In [4]:
import hydra
from hydra import compose, initialize

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path='../lerobot/configs/', version_base="1.2")
cfg = compose(config_name="default", overrides=["policy=diffusion", "env=pusht"])

device = torch.device('cuda')

In [ ]:
dataset = make_dataset(cfg)
if cfg.training.get("drop_n_last_frames"):
    shuffle = False
    sampler = EpisodeAwareSampler(
        dataset.episode_data_index,
        drop_n_last_frames=cfg.training.drop_n_last_frames,
        shuffle=True,
    )
else:
    shuffle = True
    sampler = None
dataloader = torch.utils.data.DataLoader(
    dataset,
    num_workers=cfg.training.num_workers,
    batch_size=cfg.training.batch_size,
    shuffle=shuffle,
    sampler=sampler,
    pin_memory=device.type != "cpu",
    drop_last=False,
)
dl_iter = cycle(dataloader)